### Baselines:
- BERT-based classifier trained on the data
- Some form of siamese-nn

### Ideas:
- ...

In [2]:
import json
import pandas as pd
import numpy as np
import sys

pd.set_option('display.max_colwidth', None)
sys.path.append('./src-py')

In [3]:
import sbert_training
from utils import *

In [4]:
from datasets import load_dataset, load_metric, Dataset, Split
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline, DebertaForSequenceClassification
from transformers import TrainingArguments, Trainer
import wandb
import torch
from tqdm import tqdm

In [5]:
output_path = "../../data-ceph/arguana/argmining22-sharedtask/models/"

In [5]:
taska_training_df = pd.read_csv('../data/TaskA_train.csv')
taska_valid_df = pd.read_csv('../data/TaskA_dev.csv')

In [6]:
taska_training_df = taska_training_df[taska_training_df.Novelty != 0]
taska_valid_df = taska_valid_df[taska_valid_df.Novelty != 0]

In [7]:
taska_training_df['input_txt'] = taska_training_df.apply(lambda x: '[CLS] {}:{} [SEP] {} [SEP]'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)
taska_valid_df['input_txt'] = taska_valid_df.apply(lambda x: '[CLS] {}:{} [SEP] {} [SEP]'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)

## Fine-tune the NLI model on the training data:

In [8]:
nli_tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-base-mnli')
nli_model     = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-base-mnli')
arg_stance_pipeline = TextClassificationPipeline(model=nli_model, tokenizer=nli_tokenizer, framework='pt', task='validity_classifier', device=0)

Some weights of the model checkpoint at microsoft/deberta-base-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
train_dataset = Dataset.from_pandas(taska_training_df)
eval_dataset = Dataset.from_pandas(taska_valid_df)

In [10]:
nli_model.config.id2label

{0: 'CONTRADICTION', 1: 'NEUTRAL', 2: 'ENTAILMENT'}

In [11]:
novelty_map = dict([
    (1, 1), # if novel -> neutral label
    (-1, 2) # not novel -> entailment label
])

In [12]:
inverse_novelty_map = dict([
    (2,-1),
    (1,1),
    (0,1) 
])

In [13]:
def preprocess(example):
    inputs = nli_tokenizer(example["input_txt"], add_special_tokens=False, padding=True, max_length=512)
    inputs['labels'] = list(map(novelty_map.get, example['Novelty']))
    return inputs

In [14]:
train_dataset = train_dataset.map(preprocess, batched=True)
eval_dataset = eval_dataset.map(preprocess, batched=True)

Parameter 'function'=<function preprocess at 0x7f3d0838af70> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2279: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
training_args = TrainingArguments(
    output_dir= output_path + "/task-A/novelty/classification/nli-model", 
    report_to="wandb",
    overwrite_output_dir=True,
    metric_for_best_model = 'f1',
    evaluation_strategy = 'steps',          # check evaluation metrics at each epoch
    learning_rate = 5e-5,                   # we can customize learning rate
    max_steps = 200, # five epochs
    logging_steps = 50,                    # we will log every 50 steps which is an epoch given the 700 examples and 16 batch size
    eval_steps = 50,                      # we will perform evaluation every 500 steps
    save_steps = 50,
    load_best_model_at_end = True,
)

In [17]:
trainer = Trainer(
    model=nli_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [18]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: Premise, Validity-Confidence, Novelty-Confidence, Conclusion, Validity, input_txt, Novelty, __index_level_0__, topic. If Premise, Validity-Confidence, Novelty-Confidence, Conclusion, Validity, input_txt, Novelty, __index_level_0__, topic are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 718
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation 

/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Recall,Precision,F1
50,0.504500,0.597112,0.243902,0.909091,0.384615
100,0.317600,0.727261,0.390244,0.744186,0.512000
150,0.171500,1.678869,0.146341,1.000000,0.255319
200,0.072700,1.351955,0.365854,0.857143,0.512821


The following columns in the evaluation set  don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: Premise, Validity-Confidence, Novelty-Confidence, Conclusion, Validity, input_txt, Novelty, __index_level_0__, topic. If Premise, Validity-Confidence, Novelty-Confidence, Conclusion, Validity, input_txt, Novelty, __index_level_0__, topic are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
Saving model checkpoint to ../../data-ceph/arguana/argmining22-sharedtask/models//task-A/novelty/classification/nli-model/checkpoint-50
Configuration saved in ../../data-ceph/arguana/argmining22-sharedtask/models//task-A/novelty/classification/nli-model/checkpoint-50/config.json
Model weights saved in ../../data-ceph/arguana/argmining22-sharedtask/models//task-A/novelty/classification/nli-model/checkpoint-50/pytorch_model.bin
/usr/local/l

TrainOutput(global_step=200, training_loss=0.26656726002693176, metrics={'train_runtime': 168.7093, 'train_samples_per_second': 18.968, 'train_steps_per_second': 1.185, 'total_flos': 237020932814400.0, 'train_loss': 0.26656726002693176, 'epoch': 4.44})

In [19]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: Premise, Validity-Confidence, Novelty-Confidence, Conclusion, Validity, input_txt, Novelty, __index_level_0__, topic. If Premise, Validity-Confidence, Novelty-Confidence, Conclusion, Validity, input_txt, Novelty, __index_level_0__, topic are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 1.351954698562622,
 'eval_recall': 0.36585365853658536,
 'eval_precision': 0.8571428571428571,
 'eval_f1': 0.5128205128205128,
 'eval_runtime': 0.5135,
 'eval_samples_per_second': 389.507,
 'eval_steps_per_second': 25.318,
 'epoch': 4.44}

## Fine-tune simple BERT model on the training data:

In [20]:
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
bert_model     = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://hugg

In [21]:
train_dataset = Dataset.from_pandas(taska_training_df)
eval_dataset = Dataset.from_pandas(taska_valid_df)

In [22]:
novelty_map = dict([ # avoid negative labels
    (1, 1), 
    (-1, 0)
])

In [23]:
def preprocess(example):
    inputs = bert_tokenizer(example["input_txt"], add_special_tokens=False, padding=True, truncation=True, max_length=512)
    inputs['labels'] = list(map(novelty_map.get, example['Novelty']))
    return inputs

In [24]:
train_dataset = train_dataset.map(preprocess, batched=True)
eval_dataset = eval_dataset.map(preprocess, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [26]:
training_args = TrainingArguments(
    output_dir= output_path + "/task-A/novelty/classification/bert", 
    report_to="wandb",
    overwrite_output_dir=True,
    metric_for_best_model = 'f1',
    evaluation_strategy = 'steps',          # check evaluation metrics at each epoch
    learning_rate = 5e-5,                   # we can customize learning rate
    max_steps = 200, # five epochs
    logging_steps = 50,                    # we will log every 50 steps which is an epoch given the 700 examples and 16 batch size
    eval_steps = 50,                      # we will perform evaluation every 500 steps
    save_steps = 50,
    load_best_model_at_end = True,
)

PyTorch: setting up devices


In [27]:
trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics(x, average='macro')
)

max_steps is given, it will override any value given in num_train_epochs


In [28]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Premise, Validity-Confidence, Novelty-Confidence, Conclusion, Validity, input_txt, Novelty, __index_level_0__, topic. If Premise, Validity-Confidence, Novelty-Confidence, Conclusion, Validity, input_txt, Novelty, __index_level_0__, topic are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 718
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps 

Step,Training Loss,Validation Loss,Recall,Precision,F1
50,0.488500,0.825894,0.000000,0.000000,0.000000
100,0.450800,0.838882,0.000000,0.000000,0.000000
150,0.407800,1.028969,0.000000,0.000000,0.000000
200,0.335600,0.927118,0.060976,0.625000,0.111111


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Premise, Validity-Confidence, Novelty-Confidence, Conclusion, Validity, input_txt, Novelty, __index_level_0__, topic. If Premise, Validity-Confidence, Novelty-Confidence, Conclusion, Validity, input_txt, Novelty, __index_level_0__, topic are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ../../data-ceph/arguana/argmining22-sharedtask/models//task-A/novelty/classification/bert/checkpoint-50
Configuration 

TrainOutput(global_step=200, training_loss=0.42068331718444824, metrics={'train_runtime': 141.3338, 'train_samples_per_second': 22.641, 'train_steps_per_second': 1.415, 'total_flos': 203401290234240.0, 'train_loss': 0.42068331718444824, 'epoch': 4.44})

In [29]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Premise, Validity-Confidence, Novelty-Confidence, Conclusion, Validity, input_txt, Novelty, __index_level_0__, topic. If Premise, Validity-Confidence, Novelty-Confidence, Conclusion, Validity, input_txt, Novelty, __index_level_0__, topic are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.9271178841590881,
 'eval_recall': 0.06097560975609756,
 'eval_precision': 0.625,
 'eval_f1': 0.1111111111111111,
 'eval_runtime': 0.4072,
 'eval_samples_per_second': 491.162,
 'eval_steps_per_second': 31.926,
 'epoch': 4.44}